In [1]:
import os
os.chdir("/Users/thomasmullen/VSCodeProjects/RawDatConversion")

import subprocess
from pathlib import Path
import numpy as np
import zarr
from numcodecs import Blosc
import mat73
import logging
from skimage import io as skio
from data_conversion.dat_file_functs import Struct, LoadConfig, calc_total_planes
from data_conversion.data_stream_io import map_dats_to_volume, load_dats_for_vol
from tqdm import trange


In [2]:
# parse data
# dark_plane = np.load("/Volumes/TomMullen02/20221204/dark_plane.npy")
# dat_dir=Path("/Volumes/TomMullen02/20221204/high_resolution/h2bhuc_f01_4dpf_run3_HR")
# info_path = Path(f"/Volumes/TomMullen02/20221204/high_resolution/h2bhuc_f01_4dpf_run3_HR_info.mat")
# config_path = Path(f"{dat_dir}",f"acquisitionmetadata.ini")
dark_plane = np.load("/Volumes/TomMullen/20221124/f02_6dpf_huc_h2b/dark_plane.npy")
dat_dir=Path("/Volumes/TomMullen/20221124/f02_6dpf_huc_h2b/f02_6dpf_huc_h2b_run2_HR")
info_path = Path(f"/Volumes/TomMullen/20221124/f02_6dpf_huc_h2b/f02_6dpf_huc_h2b_run2_HR_info.mat")
config_path = Path(f"{dat_dir}",f"acquisitionmetadata.ini")


In [3]:
# load daq info
daq = Struct(mat73.loadmat(info_path)).info.daq
# load config file
config=LoadConfig(config_path)
# generate dat file loader
dat_loader = config.get_dat_dimensions()
# load and sort dat spools files
total_planes = calc_total_planes(daq)
file_names = dat_loader.sort_spool_filenames(total_planes)
# prepare slices and dat vols for each camera vol
dat_vol_arrays, dat_slice_array = map_dats_to_volume(daq, dat_loader.n_planes)


['data', 'multiimage']


In [4]:
flyback = 2
hr_shape=(int(daq.numberOfScans),
            int(daq.pixelsPerLine-flyback),
            dat_loader.x_crop,
            dat_loader.y_crop)
hr_shape

(1, 999, 400, 399)

In [5]:
dark_vol = np.tile(dark_plane, (int(daq.pixelsPerLine-flyback),1,1)).astype(np.uint16)
dark_vol.shape

(999, 400, 399)

In [6]:
# load high resolution volume
volume = load_dats_for_vol(dat_loader,
                                dat_dir, 
                                file_names, 
                                dat_vol_arrays[0], 
                                dat_slice_array[0])[...,:-flyback]
# reorder indices - t, z, y, x
volume = volume.transpose(2,0,1)
# subtract background
volume+=110
volume-=dark_vol

In [7]:
volume.shape

(999, 400, 399)

In [8]:
# save the high resolution volume as a tiff
skio.imsave(Path(f"{dat_dir.parent}","high_resolution.tif"), volume.astype(np.uint16))

/var/folders/gn/k1gxpths3215w28g0srkkq7r0000gn/T/ipykernel_16703/4030091149.py:2: UserWarning: /Volumes/TomMullen/20221124/f02_6dpf_huc_h2b/high_resolution.tif is a low contrast image
  skio.imsave(Path(f"{dat_dir.parent}","high_resolution.tif"), volume.astype(np.uint16))
